# 科创中心考试答题卡

答卷要求
---
1.作答形式：以小组形式（3-4人)进行答题。可以随意在互联网寻找相关资料，不可小组之间相互抄袭或找外援。（被发现破坏公平行为直接取消全组成绩作废）前三题为技术题，需要代码实现。最后一题为简答题，也可以进行实现论证，会根据作答情况进行酌情加分。


2.提交要求：尽量以ipython文件格式（本文件）于2022年1月10日发送至邮箱2286988225@qq.com（邮件名：科创中心考试答题+小组名（自拟）/个人姓名），完成度在60%以上的团队将对结果进行展示交流。


3.卷面要求：将代码在所备注的部分进行填写。可以增加行数，但不要填到别的题的答题范围内。尽量加上注释，卷面可读性会提高小组评分。（以让阅卷人尽量能清晰明了的看懂代码为目的）。


3.在填写名字的地方备注上分工如下例子：

信息填写
---
例：答卷人1姓名：杨雨薇（第一题数据分析，第三题可视化）


答卷人1学号：2020****


答卷人2姓名：


答卷人2学号：


答卷人3姓名：


答卷人3学号：


答卷人3姓名：


答卷人3学号：

数据说明如下:
---
数据为x省x市x区x年x月x日-xx年x月x日的处罚事故数据。数据包括报警时间、报警地点等各类警情信息。某大队要根据所给出的数据，对数据进行分析，并根据该数据进行合理的路径规划（也可以根据历史数据进行预测未来一周的路径）。

第一题：
---
根据所给出的事故双方的当事人的身份证号，判断其身份证号是否合法，并对身份证号、电话号码进行解析（ id_validator，phonenumber）获取其年龄、户籍、目前通讯地址（电话号码所在地）。通过统计身份证号，以及所给人员的信息，对事故多发人员的户籍地、年龄、居住社区，并进行可视化，对高发事故人员画像。

In [1]:
# 第一题题目：
# 根据所给出的事故双方的当事人的身份证号，判断其身份证号是否合法，并对身份证号、电话号码进行解析（ id_validator，phonenumber）获取其年龄、户籍、目前通讯地址（电话号码所在地）。通过统计身份证号，以及所给人员的信息，对事故多发人员的户籍地、年龄、居住社区，并进行可视化，对高发事故人员画像。
# 
# 注释要求：函数的第一行描述函数功能，第二行写作者，第三行描述具体思路，第四行分点完成具体实现

import phone 
from id_validator import validator
import time
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, wait, ALL_COMPLETED



list1 = [134, 135, 136, 137, 138, 139, 147, 150, 151, 152, 157, 158, 159, 178, 182, 183, 184, 187, 188,
         198, 130, 131, 132, 155, 156, 185, 186, 145, 146, 166, 175, 176, 133, 153, 177, 180, 181, 189, 173, 199, 191]
list2 = [162, 165, 167, 170, 171]

# 函数功能：电话号码归属地查询（即目前通讯地址）
# 作者：胡文强,孟昊阳（加了校检）
# 主要思路：通过 phone 库来获得电话号码信息并且提取、格式化输出为所需归属地信息
# 具体实现：
# 1. 函数获取 input_phone_number, 并将使用 phone.Phone().find() 获取具体信息，将返回值存储在 returned_phone_info 中
#    样式：{'phone': '180xxxxxxxx', 'province': '四川', 'city': 'xx', 'zip_code': 'xxxxxx', 'area_code': 'xxxx', 'phone_type': '电信'}
# 2. 之后将获取格式化的信息转为 string 类型，避免类型错误
# 3. 之后用 split 以 ',' 将返回值分隔成多个部分，取第 2 部分 (从零开始计算，故取 1): 'province': '四川', 与第 3 部分：'city': 'xx',
# 4. 分别用 split 处理得到参数的具体值并去除单引号（replace()）和空格（strip()），然后相加得到最终需要的归属地信息
def get_phone_information_address(input_phone_number): #社区
    if (input_phone_number.strip() != 'NaN'):
        if len(input_phone_number) == 11 and (int(input_phone_number[:3]) in list1):
            returned_phone_info = phone.Phone().find(input_phone_number)
            returned_phone_info_address = str(returned_phone_info).split(',')[1].split(':')[1].replace("'","").strip() + str(returned_phone_info).split(',')[2].split(':')[1].replace("'","").strip()
            return returned_phone_info_address
        elif len(input_phone_number) == 11 and (int(input_phone_number[:3]) in list2):
            return 'error: 虚拟号码!'
        else:
            return 'error: 非法号码!'
    else:
        return 'error: 无输入号码!'

# 函数功能：根据身份证号查询年龄
# 作者：胡文强
# 主要思路：通过 id_validator 库获取身份证号码信息并且提取、格式化为所需的年龄信息（暂时强制转换为 str 类型的数值，按需修改）
# 具体实现：
# 1. 首先使用 validator.is_valid() 验证身份证号是否合法（valid)，不合法的话直接返回 -1 异常值
# 2. 通过 validator.get_info() 得到身份证号具体信息，将返回值存储在 returned_id_info 中
#    样式：{'address_code': 'xxxxxx', 'abandoned': 0, 'address': '四川省xxx', 'address_tree': ['四川省', 'xxx', 'xx'], 'age': 19, 'birthday_code': '2003-08-08', 'constellation': '狮子座', 'chinese_zodiac': '未羊', 'sex': 1, 'length': 18, 'check_bit': '0'}
# 3. 之后将 returned_id_info 强制转换为 string 型变量，通过 split() 使用','对字符串进行分隔，取第7部分（由0开始计算，故值为6）
# 4. 然后将提出出来的值再次 split()，取第二部分，并通过 strip() 去除空格，之后返回 string 类型的年龄值
def get_id_information_age(input_id_number): #年龄
    if validator.is_valid(str(input_id_number)) and input_id_number.strip() != 'NaN':
        returned_id_info = validator.get_info(str(input_id_number))
        returned_id_info_age = str(returned_id_info).split(',')[6].split(':')[1].strip() 
        return str(returned_id_info_age)
    elif input_id_number.strip == 'NaN':
        return ''
    else:
        return 'error: 身份证号有误，无法处理年龄!'

# 函数功能：根据身份证号查询户籍地址
# 作者：胡文强
# 主要思路：通过 id_validator 库获取身份证号码信息并且提取、格式化为所需的户籍信息（暂时强制转换为 str 类型的数值，按需修改）
# 具体实现：
# 1. 首先使用 validator.is_valid() 验证身份证号是否合法（valid)，不合法的话直接返回 -1 异常值
# 2. 通过 validator.get_info() 得到身份证号具体信息，将返回值存储在 returned_id_info 中
#    样式：{'address_code': 'xxxxxx', 'abandoned': 0, 'address': '四川省xxx', 'address_tree': ['四川省', 'xxx', 'xx'], 'age': 19, 'birthday_code': '2003-08-08', 'constellation': '狮子座', 'chinese_zodiac': '未羊', 'sex': 1, 'length': 18, 'check_bit': '0'}
# 3. 之后将 returned_id_info 强制转换为 string 型变量，通过 split() 使用','对字符串进行分隔，取第3部分（由0开始计算，故值为2）
# 4. 然后将提出出来的值再次 split()，取第二部分，由于数值存在单引号，故使用 replace() 将其替换为 NULL，并通过 strip() 去除空格，之后返回 string 类型的户籍地址
def get_id_information_address(input_id_number):
    if validator.is_valid(str(input_id_number)) and input_id_number.strip() != 'NaN':
        returned_id_info = validator.get_info(str(input_id_number))
        returned_id_info_address = str(returned_id_info).split(',')[2].split(':')[1].replace("'","").strip() 
        return returned_id_info_address
    elif input_id_number.strip == 'NaN':
        return ''
    else:
        return 'error: 身份证号有误，无法处理户籍地址!'

# 代码功能：将姓名，身份证，住址，年龄，社区等信息放在excel表格内   ——V 0.5 Demo
# 作者：孟昊阳
# 主要思路：
# 具体实现:
def main_function():
    start_time = time.time()
    # 引入考试数据
    df = pd.read_excel("./考试数据.xls")
    # 对身份证一列进行数据提取
    ID_get = df['Unnamed: 7'].to_string(header=False, index=False).split('\n')
    ID_get = ID_get[1:]
    # 下面五行是为结果建立基本已有信息：姓名+身份证号
    df_base = df['Unnamed: 7']
    df_base2 = df['Unnamed: 8']
    df_base = df_base[1:]
    df_base2 = df_base2[1:]
    # 将姓名和身份证整合到一个Datafarme（df_base3）
    df_base3 = pd.concat([df_base, df_base2], axis=1)
    ### 后期加了一个电话号码进去
    df_phone = df['Unnamed: 3']
    df_base3 = pd.concat([df_base3, df_phone], axis=1)
    df_base3 = df_base3[1:]
    df_base3 = df_base3.rename(columns={'Unnamed: 7':'身份证号码', 'Unnamed: 8':'姓名', 'Unnamed: 3':'联系人电话'})
    ##### 后期加入了表头更改以及新Darafarme的整理
    data2 = [{'身份证号码': '身份证号码', '姓名': '姓名', '联系人电话': '联系人电话'}]
    df_blank2 = pd.DataFrame(data2)
    df_base3 = df_blank2.append(df_base3, ignore_index=True)

    # 建立年龄和地址的空Datafarme框架
    AGEADDRESS_list = pd.DataFrame()
    # 利用for循环历遍每一个信息，并用函数处理分析
    for ID in (ID_get):
        AGE = get_id_information_age(ID)
        ADDRESS = get_id_information_address(ID)
        AGEADDRESS_list = AGEADDRESS_list.append(pd.DataFrame({'年龄': [AGE], '地址': [ADDRESS]}), ignore_index=True)
    # 将得到年龄地址Datafarme和之前的已有信息Datafarme进行合并

    # 提取电话号信息
    Phone_get = df['Unnamed: 3'].to_string(header=False, index=False).split('\n')
    Phone_get = Phone_get[1:]
    # 建立空Datafarme来装电话信息
    Phone_list = pd.DataFrame()
    # 同理利用for循环历遍电话号并进行处理
    for Phone_number in (Phone_get):
        i = Phone_number.replace(" ","")
        Number = get_phone_information_address(i)
        Phone_list = Phone_list.append(pd.DataFrame({'归属地': [Number]}), ignore_index=True)
    
    # 对Datafarme进行进一步整理
    data = [{'年龄': '年龄', '地址': '地址', '归属地': '归属地'}]
    df_blank = pd.DataFrame(data)
    # 将多个Datafarme进行合并处理
    AGEADDRESSPhone_list = pd.concat([AGEADDRESS_list, Phone_list], axis=1)
    AGEADDRESSPhone_list = df_blank.append(AGEADDRESSPhone_list, ignore_index=True)
    AGEADDRESSPhone_list = pd.concat([df_base3 ,AGEADDRESSPhone_list], axis=1)
    AGEADDRESSPhone_list.to_excel(r'./数据结果.xlsx', index = False, header=True)

    end_time = time.time()
    print ("Used Time: ", end_time - start_time,'s')
    # 后期维护升级内容： 1.提高运行速度，方向：利用pool池子多线程，或者，利用迭代（暂时还没查到）,或者用Numba暴力加速，但这个库只支持函数，要想办法把内容变成函数
    #                  2.加入计时器 (已完成)
    #                  3.利用numpy和matlop进行可视化（同样要考虑时间优化问题），或者直接在excel上进行图表展示
    #                  4.大部分可以简化，之后有时间再弄吧

if __name__ == "__main__":
    main_function()
    from openpyxl import load_workbook

    wb = load_workbook(r'./数据结果.xlsx')
    ws = wb[wb.sheetnames[0]]  # 打开第一个sheet
    ws.column_dimensions['A'].width = 19.0  # 调整列A宽
    ws.column_dimensions['B'].width = 8.0
    ws.column_dimensions['C'].width = 20.0
    ws.column_dimensions['D'].width = 7.0
    ws.column_dimensions['E'].width = 40.0
    ws.column_dimensions['F'].width = 13.0
    wb.save(r'./数据结果.xlsx')


Used Time:  18.398927211761475 s


第二题：
---
根据标注地点从地图API获取其具体坐标

In [58]:
# 第二题题目：
# 根据标注地点从地图API获取其具体坐标

import urllib
import json
from requests.api import request
from pandas import Series, DataFrame
import pandas as pd
import time
import matplotlib.pyplot as plt

##################################################################################
# 函数功能：调取高德地图API获得信息
# 作者：胡文强
# 主要思路：通过获取的数据，构造URL并通过requests库向高德地图 API 访问得到返回值，再进行数据处理得到有效信息
# 具体实现：
# 1. 获取值：
#   1. returned_information_format: 返回信息的类型：json 还是 xml，我觉得json更方便分析
#   2. input_x, input_y: 输入的 GPS 的 X、Y 坐标
#   3. input_key: 用户 Key
#   4. result_radius: 查询的半径
#   5. returned_information_kind: 是返回所有的结果（all）还是单个（base）
# 2. 校验值是否为 NaN，如果是则直接返回 '-2', 表示输入数据有误
# 3. 通过 urllib 的 request 来调取API得到返回值，如果正常则返回 string 类型的 returned_result，如果有异常则返回 string 类型的 '-1'
# 4. 将返回值给别的函数（get_processed_location）处理，得到所需数据
# 备注：通过孟昊阳的高德地图账号申请到 key 为：aad49afa17b46e85e060bbe252f25a80
def get_location(returned_information_format, input_x, input_y, input_key, returned_information_kind):
    # Url example: https://restapi.amap.com/v3/geocode/regeo?output=xml&location=116.310003,39.991957&key=<用户的key>&radius=1000&extensions=all (all/base)
    if (input_x.strip() != 'NaN' and input_y.strip() != 'NaN'):
        url = 'https://restapi.amap.com/v3/geocode/regeo?output=' + str(
            returned_information_format).strip() + 'xml&location=' + str(input_x).strip() + ',' + str(
            input_y).strip() + '&key=' + str(input_key).strip() + '&radius=1000' + '&extensions=' + str(
            returned_information_kind).strip()
        try:
            returned_result = urllib.request.urlopen(url).read()
            returned_result = returned_result.decode()
            returned_result = str(returned_result)
            return returned_result
        except:
            return '-1'
    else:
        return '-2'
##################################################################################

##################################################################################
# 函数功能：处理通过地图API获取到的json信息得到地址输出
# 作者：胡文强
# 主要思路：先进行校验，若收录则对信息进行处理，若无法得到任何结果则返回 “未收录”
# 具体实现：
# 1. 进行校验，若 return_result 不为 -1 （即网络正常，可访问）则开始处理：
#   1. 将 returned_result 转为json数据
#   2. 从中提取出 regeocode 中的 formatted_address 项，如果失败返回 “error: 坐标有误！”
# 2. 若为 '-1' 则返回结果 “error: 网络错误！”
# 3. 若为 '-2' 则返回结果 “error: 输入坐标不全!”
def get_processed_location(returned_result):
    if (returned_result != '-1' and returned_result != '-2'):
        try:
            temp = json.loads(returned_result)
            processed_location = temp["regeocode"]["formatted_address"]
            return processed_location
        except:
            return 'error: 坐标有误!'
    elif (returned_result == '-1'):
        return 'error: 网络错误!'
    elif (returned_result == '-2'):
        return 'error: 输入坐标不全!'
##################################################################################

##################################################################################
# 函数功能：读取表格中的数据并处理，完成后进行输出
# 作者：孟昊阳，胡文强
# 主要思路：读入表格中的GPS列信息，利用上述函数进行处理，之后输出到Excel中
# 具体实现：如代码中注释所述
if __name__ == "__main__":
    # 计时
    T1 = time.time()

    # result = get_location('json', '119.944296584982', '30.097262635875', 'aad49afa17b46e85e060bbe252f25a80', 'base')
    # print(get_processed_location(result))
    # 引入考试数据
    df = pd.read_excel("./考试数据.xls")  # 被胡文强改为了相对路径
    # 对GPS两列进行数据提取
    GPS_get_x = df['GPS_X'].to_string(header=False, index=False).split('\n')
    GPS_get_x = GPS_get_x[1:]

    GPS_get_y = df['GPS_Y'].to_string(header=False, index=False).split('\n')
    GPS_get_y = GPS_get_y[1:]
    # AGEADDRESS_list = pd.DataFrame()#建造一个空列表
    output_list = pd.DataFrame()
    # 将GPS提取并处理得到具体地址
    step_counter = 1
    for GPS_x, GPS_y in zip(GPS_get_x, GPS_get_y):
        GPS = get_processed_location(get_location('json', GPS_x, GPS_y, 'aad49afa17b46e85e060bbe252f25a80', 'base'))
        output_list = output_list.append(pd.DataFrame({'GPS地址': [GPS]}), ignore_index=True)
        print(step_counter,": ", GPS)
        step_counter = step_counter +1


    data3 =  [{'GPS地址': 'GPS地址'}]
    df_blank3 = pd.DataFrame(data3)
    output_list = df_blank3.append(output_list, ignore_index=True)

##################################################################################
    df_base = df['Unnamed: 7']
    df_base2 = df['Unnamed: 8']
    df_base = df_base[1:]
    df_base2 = df_base2[1:]
    df_date = df['报警时间']
    df_date = df_date[1:]
    GPS_X = df['GPS_X']
    GPS_X = GPS_X[1:]
    GPS_Y = df['GPS_Y']
    GPS_Y = GPS_Y[1:]
    df_GPS = pd.concat([GPS_X, GPS_Y], axis=1)

    df_base3 = pd.concat([df_base, df_base2], axis=1)

    df_phone = df['Unnamed: 3']
    df_base3 = pd.concat([df_base3, df_phone], axis=1)
    df_base3 = df_base3[1:]
    df_base3 = pd.concat([df_date, df_base3], axis=1)
    df_base3 = pd.concat([df_base3, df_GPS], axis=1)
    df_base3 = df_base3.rename(columns={'报警时间': '报警时间', 'Unnamed: 7': '身份证号码', 'Unnamed: 8': '姓名', 'Unnamed: 3': '联系人电话', 'GPS_X': 'GPS_x', 'GPS_Y': 'GPS_y'})

    data2 = [{'报警时间': '报警时间', '身份证号码': '身份证号码', '姓名': '姓名', '联系人电话': '联系人电话', 'GPS_x': 'GPS_x', 'GPS_y': 'GPS_y'}]
    df_blank2 = pd.DataFrame(data2)
    df_base3 = df_blank2.append(df_base3, ignore_index=True)
    output_list = pd.concat([df_base3, output_list], axis=1)

##################################################################################

    output_list.to_excel(r'./GPS数据结果.xlsx', index=False, header=True)
##################################################################################
    ###设置excel表格格式
    from openpyxl import load_workbook

    wb = load_workbook(r'./GPS数据结果.xlsx')
    ws = wb[wb.sheetnames[0]]  # 打开第一个sheet
    ws.column_dimensions['A'].width = 31.0  # 调整列A宽
    ws.column_dimensions['B'].width = 19.0
    ws.column_dimensions['C'].width = 8.0
    ws.column_dimensions['D'].width = 15.0
    ws.column_dimensions['E'].width = 20.0
    ws.column_dimensions['F'].width = 20.0
    ws.column_dimensions['G'].width = 80.0
    wb.save(r'./GPS数据结果.xlsx')
    # 计时
    T2 = time.time()

    print('程序运行时间:%s秒' % (T2 - T1))
##################################################################################


第三题：
---
根据第二题所获取的地理坐标信息，对2020年12月1日的事故处理路线，基于案件的警情以及实际坐标进行合理的路径规划（最好能用可视化方式展现合理的规划方案）。（假设处理每起事故需5分钟）

In [59]:
###第三题答案代码

第四题：
---
根据以上所给数据给出你们觉得可行的数据分析方向，方法，以及可行的技术路线，理论依据。（如果实现可加分）

第四题答题区域：